# New Baseline Exploration

## What this code does

This is a script for a baseline, to get you started. It just calculates the mean probabilites of dangerous objects for all zones from the training samples and submits these. 

This approach gives 0.29 of log-loss on the training set. 

In [1]:
# imports
from __future__ import print_function
from __future__ import division
import numpy as np
import os
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as stats

from sklearn import *
import sklearn
import pandas as pd
import numpy as np
import xgboost as xgb

C:\Users\jehun\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\jehun\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\jehun\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\jehun\Anacon

In [2]:
train = pd.read_csv('D:\Backups\KaggleJul2017\TSA\stage1_labels.csv') 
test = pd.read_csv('D:\Backups\KaggleJul2017\TSA\stage1_sample_submission.csv')

trainp = train.copy()
trainp['Id'] = trainp['Id'].map(lambda x: x.split('_')[1])

piv = pd.pivot_table(trainp, columns='Id', values='Probability', aggfunc='mean', fill_value=0)
d = pd.DataFrame.to_dict(piv)
#from https://www.kaggle.com/philippsp/baseline-lb-0-29089
#d['Zone9']['Probability'] = 0.05

y = train['Probability'].values
pid = test['Id'].values

df_all = pd.concat((train, test), axis=0, ignore_index=True)



df_all['len'] = df_all['Id'].map(len)

for i in range(38):
    df_all['c'+str(i)] = df_all['Id'].map(lambda x: str(x[i]))
df_all['c38'] = df_all['Id'].map(lambda x: str(x[i]) if len(x)==39 else '')
df_all = df_all.drop(['Id','Probability','len'], axis=1)




for c in df_all.columns:
    if df_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        df_all[c] = lbl.fit_transform(df_all[c].values)
        #print(c, len(df_all[c].unique()))

train = df_all.iloc[:len(train)]
test = df_all.iloc[len(train):]

params = {
    'eta': 0.02,
    'max_depth': 5,
    'objective': 'reg:linear',
    'eval_metric': 'logloss',
    'seed': 12,
    'silent': True
}

fold = 5
for i in range(fold):
    x1, x2, y1, y2 = model_selection.train_test_split(train, y, test_size=0.18, random_state=i)
    watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=50)
    if i != 0:
        pred += model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
        
pred /= fold

submission = pd.DataFrame(pred, columns=['Probability'])
submission['Id'] = pid
submission['Probability'] += submission['Id'].map(lambda x: d[str(x).split('_')[1]]['Probability'])*3
submission['Probability'] /= 4
submission.to_csv('submission_xgb.csv', index=False)

[0]	train-logloss:0.680101	valid-logloss:0.680365
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[50]	train-logloss:0.379441	valid-logloss:0.391032
[100]	train-logloss:0.314986	valid-logloss:0.338946
[150]	train-logloss:0.296451	valid-logloss:0.332806
Stopping. Best iteration:
[147]	train-logloss:0.297088	valid-logloss:0.332727

[0]	train-logloss:0.680108	valid-logloss:0.680391
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[50]	train-logloss:0.37926	valid-logloss:0.389851
[100]	train-logloss:0.315098	valid-logloss:0.336328
[150]	train-logloss:0.297125	valid-logloss:0.329098
[200]	train-logloss:0.289033	valid-logloss:0.330224
Stopping. Best iteration:
[155]	train-logloss:0.296006	valid-logloss:0.328996

[0]	train-logloss:0.680179	valid-logloss:0.680241
Multiple eval metrics have be

In the above code, the author used his psychological knowledge to predict that images with dangerous objects in Zone9 are underrepresented in the test set (you can guess why when you look at where the zone is ;-)), and accordingly reduce their probability.

## Problem with this approach

A major problem of this approach is that it ignores entirely the contents of the .aps files (or any other formats). 

The training and test sets, under the above approach, consists of the encoded bits of the image label, such as `00360f79fd6e02781457eda48f85da90_Zone1`. This is obviously not a valid approach. 